In [3]:
import json

# Load the dataset
with open("clinc150_uci/data_full.json", "r") as file:
    data = json.load(file)


In [4]:
# Extracting data
train_data = data['train']
val_data = data['val']
test_data = data['test']

oos_train_data = data['oos_train']
oos_val_data = data['oos_val']
oos_test_data = data['oos_test']

# Get sentences and labels
train_sentences = [item[0] for item in train_data]
train_labels = [item[1] for item in train_data]

val_sentences = [item[0] for item in val_data]
val_labels = [item[1] for item in val_data]

test_sentences = [item[0] for item in test_data]
test_labels = [item[1] for item in test_data]

oos_train_sentences = [item[0] for item in oos_train_data]
oos_val_sentences = [item[0] for item in oos_val_data]
oos_test_sentences = [item[0] for item in oos_test_data]

# Check the number of samples in each subset
len(train_sentences), len(val_sentences), len(test_sentences), len(oos_train_sentences), len(oos_val_sentences), len(oos_test_sentences)


(15000, 3000, 4500, 100, 100, 1000)

In [5]:
from transformers import RobertaTokenizer, RobertaModel
import torch

In [6]:
# Initialize tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
if torch.cuda.is_available():
    model = model.to('cuda')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def get_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=256)
    
    # Move inputs to GPU if available
    if torch.cuda.is_available():
        for key in inputs:
            inputs[key] = inputs[key].to('cuda')
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Move embeddings back to CPU if they were on GPU
    embeddings = outputs.last_hidden_state.mean(dim=1)
    if torch.cuda.is_available():
        embeddings = embeddings.cpu()
    
    return embeddings.numpy()

In [8]:
train_embeddings = get_embeddings(train_sentences)
val_embeddings = get_embeddings(val_sentences)
test_embeddings = get_embeddings(test_sentences)
oos_train_embeddings = get_embeddings(oos_train_sentences)
oos_val_embeddings = get_embeddings(oos_val_sentences)
oos_test_embeddings = get_embeddings(oos_test_sentences)

In [9]:
oos_test_embeddings.shape

(1000, 768)

In [26]:
import numpy as np
from scipy.linalg import inv

In [19]:
mean = np.mean(train_embeddings, axis=0)
covariance = np.cov(train_embeddings, rowvar=False)
cov_inverse = inv(covariance)


In [20]:
def mahalanobis(x, mean, cov_inverse):
    delta = x - mean
    return np.dot(np.dot(delta, cov_inverse), delta.T)

In [22]:
test_mahalanobis = [mahalanobis(embedding, mean, cov_inverse) for embedding in test_embeddings]
oos_test_mahalanobis = [mahalanobis(embedding, mean, cov_inverse) for embedding in oos_test_embeddings]

In [23]:
test_mahalanobis[:5], oos_test_mahalanobis[:5]

([1544.1246716934256,
  663.8241205392405,
  1094.3538775008637,
  562.2457204030361,
  659.604276040569],
 [788.6236793857533,
  1639.5125718298368,
  1261.2153645055369,
  531.1102627911605,
  640.0282306554727])

In [27]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# Label the data: in-distribution (0) and out-of-distribution (1)
y_true = [0] * len(test_mahalanobis) + [1] * len(oos_test_mahalanobis)
y_scores = test_mahalanobis + oos_test_mahalanobis

# Compute AUROC and AUPR
auroc = roc_auc_score(y_true, y_scores)
aupr = average_precision_score(y_true, y_scores)

# Compute FPR@95
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
index = np.where(tpr >= 0.95)[0][0]
fpr_95 = fpr[index]

#OOD is positive class
auroc, aupr, fpr_95


(0.7214566666666666, 0.28372634166890487, 0.6653333333333333)

In [28]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# Label the data: in-distribution (1) and out-of-distribution (0)
y_true = [1] * len(test_mahalanobis) + [0] * len(oos_test_mahalanobis)
y_scores = test_mahalanobis + oos_test_mahalanobis

# Compute AUROC
auroc = roc_auc_score(y_true, y_scores)

# Compute AUPR for in-distribution class
aupr_in = average_precision_score(y_true, y_scores)

# Compute FPR@95 for in-distribution class
fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
index = np.where(tpr >= 0.95)[0][0]
fpr_95 = fpr[index]

auroc, aupr_in, fpr_95

(0.27854333333333336, 0.7454464575815867, 0.999)

In [29]:
0.7214566666666666+0.27854333333333336

1.0

In [31]:
from sklearn.neighbors import NearestNeighbors

# Initialize and fit the Nearest Neighbors model
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(train_embeddings)

# Find distances and indices of the nearest neighbors for the test embeddings
distances, indices = nbrs.kneighbors(test_embeddings)

# Extract the nearest neighbor distances
nn_distances = distances[:, 0]

# For OOD samples
oos_distances, _ = nbrs.kneighbors(oos_test_embeddings)
oos_nn_distances = oos_distances[:, 0]

# Combine in-distribution and OOD distances and labels for evaluation
all_distances = np.concatenate([nn_distances, oos_nn_distances])
y_true = [1] * len(nn_distances) + [0] * len(oos_nn_distances)

# Compute AUROC
auroc = roc_auc_score(y_true, all_distances)

# Compute AUPR for in-distribution class
aupr_in = average_precision_score(y_true, all_distances)

# Compute FPR@95 for in-distribution class
fpr, tpr, thresholds = roc_curve(y_true, all_distances, pos_label=1)
index = np.where(tpr >= 0.95)[0][0]
fpr_95 = fpr[index]

auroc, aupr_in, fpr_95


(0.26018444444444444, 0.7330483406864483, 1.0)

In [34]:
from sklearn.neighbors import NearestNeighbors

# Initialize and fit the Nearest Neighbors model
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(train_embeddings)

# Find distances of the nearest neighbors for the test embeddings
distances, _ = nbrs.kneighbors(test_embeddings)
nn_distances = distances[:, 0]

# For OOD samples
oos_distances, _ = nbrs.kneighbors(oos_test_embeddings)
oos_nn_distances = oos_distances[:, 0]

# Label the data: in-distribution (0) and out-of-distribution (1)
y_true = [0] * len(nn_distances) + [1] * len(oos_nn_distances)
y_scores = list(nn_distances) + list(oos_nn_distances)

# Compute AUROC
auroc = roc_auc_score(y_true, y_scores)

# Compute AUPR for the positive class (OOD)
aupr_ood = average_precision_score(y_true, y_scores)

# Compute FPR@95 for the positive class (OOD)
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
index = np.where(tpr >= 0.95)[0][0]
fpr_95 = fpr[index]

auroc, aupr_ood, fpr_95


(0.7398155555555554, 0.30473436865553233, 0.6473333333333333)